In [1]:
!pip install onnx tensorflow tensorflow-addons onnx-tf torch torchvision numpy Pillow tensorflowjs[wizard]

     |████████████████████████████████| 12.8 MB 26.0 MB/s 
     |████████████████████████████████| 1.1 MB 65.6 MB/s 
     |████████████████████████████████| 226 kB 72.7 MB/s 
     |████████████████████████████████| 77 kB 7.8 MB/s 
     |████████████████████████████████| 462 kB 71.4 MB/s 
     |████████████████████████████████| 40 kB 6.7 MB/s 
     |████████████████████████████████| 248 kB 52.6 MB/s 
  Created wheel for PyInquirer: filename=PyInquirer-1.0.3-py3-none-any.whl size=32863 sha256=c719561ea1e43222218c6d81d9eec56f1fb0136a7cacb3a758fd00ddc027ec5f
  Stored in directory: /root/.cache/pip/wheels/89/3b/7b/8b3cc8ac47137eabaeb6937a3ff0d33e78a12e2ba1e3ad4ba1
Successfully built PyInquirer
  Attempting uninstall: prompt-toolkit
    Found existing installation: prompt-toolkit 1.0.18
    Uninstalling prompt-toolkit-1.0.18:
      Successfully uninstalled prompt-toolkit-1.0.18
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:


In [2]:
import collections
import IPython
import numpy as np
import onnx
import onnx_tf.backend
import PIL
import tensorflowjs
import torch
import torchvision
import tqdm.auto as tqdm

In [3]:
class Net(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = torch.nn.Sequential(collections.OrderedDict([
            ("conv0", torch.nn.Conv2d(1, 10, 5)),
            ("pool0", torch.nn.MaxPool2d(2)),
            ("relu", torch.nn.ReLU(inplace=True)),
            ("conv1", torch.nn.Conv2d(10, 20, 5)),
            ("dropout", torch.nn.Dropout2d(inplace=True)),
            ("pool1", torch.nn.MaxPool2d(2)),
            ("relu", torch.nn.ReLU(inplace=True)),
            ("flatten", torch.nn.Flatten()),
            ("fc0", torch.nn.Linear(320, 50)),
            ("relu", torch.nn.ReLU(inplace=True)),
            ("fc_dropout", torch.nn.Dropout(inplace=True)),
            ("fc1", torch.nn.Linear(50, 10)),
            ("logsoftmax", torch.nn.LogSoftmax(dim=1))
        ]))
        self.conv0 = torch.nn.Conv2d(1, 10, 5)
        self.conv1 = torch.nn.Conv2d(10, 20, 5)
        self.drop = torch.nn.Dropout2d()
        self.flatten = torch.nn.Flatten()
        self.fc0 = torch.nn.Linear(320, 50)
        self.fc1 = torch.nn.Linear(50, 10)
        self.act = torch.nn.ReLU(inplace=False)
        self.pool = torch.nn.MaxPool2d(2)
        self.out_act = torch.nn.LogSoftmax(1)

    def forward(self, x):
        return self.layers(x)

    def foobar(self, x):
        return self.forward(x), torch.randn(x.size())

In [4]:
def train(model, device, loader, optimizer, criterion, epochs):
    losses = []
    model.train()
    for epoch in tqdm.tqdm(range(epochs), desc="Training", unit="epoch", unit_scale=False):
        batch_losses = []
        for data, target in tqdm.tqdm(loader, desc=f"Epoch {epoch + 1} / {epochs}", unit="batches", unit_scale=False):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            batch_losses.append(loss.item())
        losses.append(np.mean(batch_losses))
    return losses

def eval(model, device, loader, criterion):
    model.eval()
    losses = []
    correct = 0
    with torch.no_grad():
        for data, target in tqdm.tqdm(loader, desc="Validating", unit="batches", unit_scale=False):

            
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target).item()
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
            losses.append(loss)
    return np.mean(losses), correct / len(loader.dataset)

In [5]:
BATCH_SIZE = 64
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
train_loader = torch.utils.data.DataLoader(
    torchvision.datasets.MNIST("../data", 
    train=True, download=True, 
    transform=torchvision.transforms.Compose([
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize((0.1307,), (0.3081,))
])), batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    torchvision.datasets.MNIST("../data",
    train=False,
    transform=torchvision.transforms.Compose([
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize((0.1307,), (0.3081,))
    ])), batch_size=BATCH_SIZE, shuffle=False)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



In [7]:
next(iter(train_loader))[0].size()

torch.Size([64, 1, 28, 28])

In [8]:
model = Net().to(DEVICE)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.5)
criterion = torch.nn.NLLLoss()

In [9]:
train_losses = train(model, DEVICE, train_loader, optimizer, criterion, 21)
val_loss, accuracy = eval(model, DEVICE, test_loader, criterion)

Training:   0%|          | 0/21 [00:00<?, ?epoch/s]

Epoch 1 / 21:   0%|          | 0/938 [00:00<?, ?batches/s]

Epoch 2 / 21:   0%|          | 0/938 [00:00<?, ?batches/s]

Epoch 3 / 21:   0%|          | 0/938 [00:00<?, ?batches/s]

Epoch 4 / 21:   0%|          | 0/938 [00:00<?, ?batches/s]

...

Epoch 18 / 21:   0%|          | 0/938 [00:00<?, ?batches/s]

Epoch 19 / 21:   0%|          | 0/938 [00:00<?, ?batches/s]

Epoch 20 / 21:   0%|          | 0/938 [00:00<?, ?batches/s]

Epoch 21 / 21:   0%|          | 0/938 [00:00<?, ?batches/s]

  0%|          | 0/157 [00:00<?, ?it/s]

In [10]:
torch.save(model.state_dict(), "mnist.pth")
trained_model = Net()
trained_model.load_state_dict(torch.load("mnist.pth"))

<All keys matched successfully>

In [11]:
dummy_input = torch.autograd.Variable(torch.randn(1, 1, 28, 28))
torch.onnx.export(trained_model, dummy_input, "mnist.onnx")
model = onnx.load("mnist.onnx")
tf_rep = onnx_tf.backend.prepare(model)

In [12]:
print("Image")
img = PIL.Image.open("../data/two.jpeg").resize((28, 28)).convert("L")
display(img)

Image


In [13]:
output = tf_rep.run(np.asarray(img, dtype=np.float32)[np.newaxis, np.newaxis, :, :])
print("Classified as: ", np.argmax(output))

Classified as:  8


In [14]:
tf_rep.export_graph("mnist.pb")

INFO:tensorflow:Assets written to: mnist.pb/assets


INFO:tensorflow:Assets written to: mnist.pb/assets


In [17]:
!tensorflowjs_converter --control_flow_v2=True --input_format=tf_saved_model --metadata= --saved_model_tags=serve --signature_name=serving_default --strip_debug_ops=True --weight_shard_size_bytes=4194304 mnist.pb mnist.js.pb

2022-05-20 21:02:31.638574: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Writing weight file mnist.js.pb/model.json...
